In [1]:
import pandas as pd
import numpy as np

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
AB = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosAB_202309.xlsx', dtype={'Código DANE de Municipio': str})
AB


/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Combatientes Sin Información,Otros Grupos Armados Organizados,Total Combatientes de Grupos Armados Organizados,Total Combatientes,Personas Sin Información,Total Civiles,Ventaja Militar,Total de Víctimas del Caso,Latitud,Longitud
0,1,CR000497,2006,3,12,27073,BAGADO,CHOCO,ATRATO,COMBATE Y/O CONTACTO ARMADO,...,0,0,0,0,0,1,GRUPOS ARMADOS ORGANIZADOS,1,5.410555,-76.415293
1,10,CR080150,0,0,0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,OPERACIÓN MILITAR,...,0,0,1,1,0,0,FUERZAS ARMADAS ESTATALES,1,4.000000,-72.000000
2,100,NaN,0,0,0,05854,VALDIVIA,ANTIOQUIA,NORTE DE ANTIOQUIA,COMBATIENTE MUERTO EN CIRCUNSTANCIA NO DETERMI...,...,0,0,2,2,0,0,GRUPOS ARMADOS ORGANIZADOS,2,7.165340,-75.439198
3,1000,CR044183,1966,1,31,41676,SANTA MARIA,HUILA,NORTE DEL HUILA,COMBATE Y/O CONTACTO ARMADO,...,0,0,6,6,0,0,GRUPOS ARMADOS ORGANIZADOS,6,2.939755,-75.586116
4,10000,NaN,1996,2,17,85010,AGUAZUL,CASANARE,PIEDEMONTE LLANERO,COMBATE Y/O CONTACTO ARMADO,...,0,0,2,2,0,0,FUERZAS ARMADAS ESTATALES,2,5.100675,-72.458524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37527,9995,NaN,1996,2,13,50577,PUERTO LLERAS,META,ARIARI GUAYABERO,HOSTIGAMIENTO,...,0,0,0,0,0,1,GRUPOS ARMADOS ORGANIZADOS,1,3.016222,-73.369906
37528,9996,NaN,1996,2,14,54003,ABREGO,NORTE DE SANTANDER,PROVINCIA DE RICAURTE,COMBATE Y/O CONTACTO ARMADO,...,0,0,0,0,0,0,FUERZAS ARMADAS ESTATALES,0,8.081784,-73.221734
37529,9997,NaN,1996,2,15,05837,TURBO,ANTIOQUIA,URABA,COMBATE Y/O CONTACTO ARMADO,...,0,0,2,2,0,0,FUERZAS ARMADAS ESTATALES,2,7.996898,-76.632247
37530,9998,NaN,1996,2,15,23660,SAHAGUN,CORDOBA,NORTE DE CORDOBA,OPERACIÓN MILITAR,...,0,0,0,1,0,0,GRUPOS ARMADOS ORGANIZADOS,1,8.943125,-75.445920


In [3]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
AB_filtered = AB[relevant_columns]

# Group by the required keys
CasosAB = AB_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosAB[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosAB.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,82,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1958,5,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1959,2,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1960,17,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1961,7,0,0,0,0,0


In [4]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Grupo Armado 2')
AB_filtered = AB[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    AB_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    AB_filtered.loc[AB_filtered['Grupo Armado 2'] == original, new] = AB_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelAB = AB_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()


/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3414204006.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AB_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3414204006.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AB_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3414204006.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [5]:
AP = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosAP_202309.xlsx', dtype={'Código DANE de Municipio': str})
AP

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Grupo Armado 1,...,Combatientes Sin Información,Otros Grupos Armados Organizados,Total Combatientes de Grupos Armados Organizados,Total Combatientes,Personas Sin Información,Total Civiles,Ventaja Militar,Total de Víctimas del Caso,Latitud,Longitud
0,10902,CR020590,1997,1,28,27073,BAGADO,CHOCO,ATRATO,AGENTE DEL ESTADO,...,0,0,0,6,0,0,GRUPOS ARMADOS ORGANIZADOS,6,5.410555,-76.415293
1,11932,CR201711932,1997,12,11,68464,MOGOTES,SANTANDER,PROVINCIA DE GUANENTA,AGENTE DEL ESTADO,...,0,0,0,3,0,3,GRUPOS ARMADOS ORGANIZADOS,6,6.475210,-72.969814
2,13041,CR009075,1999,3,16,15790,TASCO,BOYACA,ALTIPLANO CUNDIBOYACENSE,AGENTE DEL ESTADO,...,0,0,0,0,0,0,GRUPOS ARMADOS ORGANIZADOS,0,5.909821,-72.781011
3,13199,CR007155,1999,5,21,54720,SARDINATA,NORTE DE SANTANDER,CATATUMBO,AGENTE DEL ESTADO,...,0,0,0,0,0,1,GRUPOS ARMADOS ORGANIZADOS,1,8.113765,-72.850982
4,13865,CR000186,1999,12,25,73686,SANTA ISABEL,TOLIMA,NORTE DEL TOLIMA,AGENTE DEL ESTADO,...,0,0,0,0,0,1,GRUPOS ARMADOS ORGANIZADOS,1,4.713606,-75.097934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,618443,NaN,1988,11,12,05736,SEGOVIA,ANTIOQUIA,NORDESTE ANTIOQUEÑO,AGENTE DEL ESTADO,...,0,0,0,0,0,0,GRUPOS ARMADOS ORGANIZADOS,0,7.079640,-74.701591
489,649082,NaN,1969,12,10,05138,CAÑASGORDAS,ANTIOQUIA,OCCIDENTE ANTIOQUEÑO,AGENTE DEL ESTADO,...,0,0,0,0,0,1,GRUPOS ARMADOS ORGANIZADOS,1,6.759478,-76.032210
490,6852,CR003094,1992,12,14,54800,TEORAMA,NORTE DE SANTANDER,CATATUMBO,AGENTE DEL ESTADO,...,0,0,0,3,0,0,GRUPOS ARMADOS ORGANIZADOS,3,8.436377,-73.287409
491,8127,CR003102,1994,3,20,05038,ANGOSTURA,ANTIOQUIA,NORTE DE ANTIOQUIA,AGENTE DEL ESTADO,...,0,0,4,5,0,0,FUERZAS ARMADAS ESTATALES,5,6.885175,-75.335116


In [6]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
AP_filtered = AP[relevant_columns]

# Group by the required keys
CasosAP = AP_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosAP[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosAP.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,2,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1981,6,0,0,0,0,0
2,05031,AMALFI,ANTIOQUIA,NORDESTE ANTIOQUEÑO,1993,7,0,0,0,0,0
3,05034,ANDES,ANTIOQUIA,SUROESTE ANTIOQUEÑO,1986,2,0,0,0,0,0
4,05038,ANGOSTURA,ANTIOQUIA,NORTE DE ANTIOQUIA,1994,5,0,0,0,0,0


In [7]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Grupo Armado 2')
AP_filtered = AP[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    AP_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    AP_filtered.loc[AP_filtered['Grupo Armado 2'] == original, new] = AP_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelAP = AP_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/903753099.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AP_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/903753099.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AP_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/903753099.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [8]:
AS = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosAS_202309.xlsx', dtype={'Código DANE de Municipio': str})
AS


/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Grafitis Letreros,Vínculos Familiares,Mujeres Embarazadas,Total de Víctimas del Caso,Tipo de Armas,Latitud,Longitud
0,100000,NaN,2003,10,27,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,DESCONOCIDA,...,0,NaN,NaN,0,0,0,1,ND,4.338481,-74.366334
1,100001,NaN,2003,11,24,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,DESCONOCIDA,...,0,NaN,NaN,0,0,0,3,ND,4.338481,-74.366334
2,100002,NaN,2004,1,1,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,SICARIATO,...,1,NaN,NaN,0,0,0,1,ARMA CORTO PUNZANTE - ARMA DE FUEGO,4.338481,-74.366334
3,100003,NaN,2004,3,7,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,RETENCIÓN/EJECUCIÓN,...,0,NaN,NaN,0,0,0,1,ND,4.338481,-74.366334
4,100004,NaN,2004,3,8,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,DESCONOCIDA,...,0,NaN,NaN,0,0,0,1,ND,4.338481,-74.366334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154781,99994,NaN,2003,5,7,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,ASALTO,...,0,NaN,NaN,0,0,0,1,ARMA DE FUEGO,4.338481,-74.366334
154782,99995,NaN,2003,6,11,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,ENGAÑO,...,0,NaN,NaN,0,0,0,1,OBJETO CONTUNDENTE,4.338481,-74.366334
154783,99996,NaN,2003,8,3,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,ATAQUE INDISCRIMINADO,...,0,NaN,NaN,0,0,0,1,ARMA DE FUEGO,4.338481,-74.366334
154784,99997,NaN,2003,9,15,25290,FUSAGASUGA,CUNDINAMARCA,SUMAPAZ,RETENCIÓN/EJECUCIÓN,...,2,NaN,NaN,0,0,0,1,ARMA DE FUEGO,4.338481,-74.366334


In [9]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
AS_filtered = AS[relevant_columns]

# Group by the required keys
CasosAS = AS_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosAS[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosAS.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,163,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1958,4,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1959,7,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1960,3,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1961,8,0,0,0,0,0


In [10]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
AS_filtered = AS[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    AS_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    AS_filtered.loc[AS_filtered['Presunto Responsable'] == original, new] = AS_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelAS = AS_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3677516628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AS_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3677516628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AS_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3677516628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [11]:
AT = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosAT_202309.xlsx', dtype={'Código DANE de Municipio': str})
AT

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Pillaje,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total Combatientes,Total Civiles,Total de Víctimas del Caso,Tipo de Armas,Latitud,Longitud
0,185266,CR004878,1977,1,1,11001,"BOGOTA, D.C.","BOGOTA, D. C.",AREA METROPOLITANA DE BOGOTA,ATAQUE A OBJETIVO CIVIL CON EFECTO INDISCRIMINADO,...,0,0,NaN,NaN,0,0,0,ARTEFACTO EXPLOSIVO,4.650173,-74.107359
1,185267,NaN,1978,8,13,18001,FLORENCIA,CAQUETA,FLORENCIA Y AREA DE INFLUENCIA,ATAQUE A OBJETIVO CIVIL CON EFECTO INDISCRIMINADO,...,0,0,NaN,NaN,0,0,0,ARTEFACTO EXPLOSIVO,1.617923,-75.608560
2,185268,CR012154,1982,8,1,76001,CALI,VALLE DEL CAUCA,SUR DEL VALLE,ATAQUE A OBJETIVO MILITAR CON EFECTO INDISCRIM...,...,0,0,NaN,NaN,0,0,0,ARTEFACTO EXPLOSIVO,3.414414,-76.521565
3,185269,NaN,1991,6,1,17001,MANIZALES,CALDAS,EJE CAFETERO,ATAQUE A OBJETIVO CIVIL CON EFECTO INDISCRIMINADO,...,0,0,NaN,NaN,0,4,4,ARTEFACTO EXPLOSIVO,5.057688,-75.491049
4,185271,CR005155,1989,11,27,25754,SOACHA,CUNDINAMARCA,AREA METROPOLITANA DE BOGOTA,ATAQUE A OBJETIVO CIVIL CON EFECTO INDISCRIMINADO,...,0,0,NaN,NaN,0,107,107,ARTEFACTO EXPLOSIVO,4.585492,-74.214528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,722308,NaN,2022,3,26,11001,"BOGOTA, D.C.","BOGOTA, D. C.",AREA METROPOLITANA DE BOGOTA,ATAQUE A OBJETIVO MILITAR CON EFECTO INDISCRIM...,...,0,0,NaN,NaN,0,2,2,ARTEFACTO EXPLOSIVO,4.650173,-74.107359
258,722369,CR2017722369,2022,7,26,20383,LA GLORIA,CESAR,MAGDALENA MEDIO,ATAQUE A OBJETIVO MILITAR CON EFECTO INDISCRIM...,...,0,0,NaN,NaN,0,0,0,ARMA DE FUEGO - ARTEFACTO EXPLOSIVO,8.619298,-73.803210
259,722399,NaN,2022,7,30,54810,TIBU,NORTE DE SANTANDER,CATATUMBO,ATAQUE A OBJETIVO MILITAR CON EFECTO INDISCRIM...,...,0,0,NaN,NaN,0,0,0,ARTEFACTO EXPLOSIVO,8.639549,-72.735919
260,732450,NaN,2022,7,17,19532,PATIA,CAUCA,PATIA,ATAQUE A OBJETIVO MILITAR CON EFECTO INDISCRIM...,...,0,0,NaN,NaN,0,1,1,ARTEFACTO EXPLOSIVO,2.115670,-76.980979


In [12]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
AT_filtered = AT[relevant_columns]

# Group by the required keys
CasosAT = AT_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosAT[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosAT.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1983,0,0,0,0,0,0
1,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1985,0,0,0,0,0,0
2,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1990,14,0,0,0,0,0
3,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1992,9,0,0,0,0,0
4,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1995,24,0,0,0,0,0


In [13]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
AT_filtered = AT[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    AT_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    AT_filtered.loc[AT_filtered['Presunto Responsable'] == original, new] = AT_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelAT = AT_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1321388456.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AT_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1321388456.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AT_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1321388456.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [14]:
DF = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosDF_202309.xlsx', dtype={'Código DANE de Municipio': str})
DF

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Extorsión,Lesionados Civiles,Pillaje,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total de Víctimas del Caso,Tipo de Armas,Latitud,Longitud
0,100265,NaN,1991,5,25,25368,JERUSALEN,CUNDINAMARCA,SUROCCIDENTE DE CUNDINAMARCA,DESCONOCIDA,...,0,0,0,0,NaN,NaN,1,ND,4.562308,-74.695419
1,100282,NaN,2004,12,2,25386,LA MESA,CUNDINAMARCA,SUROCCIDENTE DE CUNDINAMARCA,RETENCIÓN,...,0,0,0,1,NaN,NaN,1,ARMA DE FUEGO,4.631044,-74.461545
2,101616,CR000088,1993,3,9,25885,YACOPI,CUNDINAMARCA,MAGDALENA MEDIO,ASALTO,...,0,0,0,3,NaN,NaN,2,ARMA CORTO PUNZANTE - ARMA DE FUEGO,5.459290,-74.338024
3,102675,NaN,1997,5,15,27615,RIOSUCIO,CHOCO,URABA,ASALTO,...,0,0,0,0,NaN,NaN,1,ND,7.436632,-77.113110
4,103705,NaN,2006,11,11,41396,LA PLATA,HUILA,SUR DEL HUILA,DESCONOCIDA,...,0,0,0,0,NaN,NaN,2,ND,2.389647,-75.891451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68597,98477,CR037072,1990,3,16,23580,PUERTO LIBERTADOR,CORDOBA,ALTO SINU Y SAN JORGE,DESCONOCIDA,...,0,0,0,0,NaN,NaN,1,ND,7.888835,-75.671800
68598,98981,NaN,2002,2,25,23807,TIERRALTA,CORDOBA,URABA,RETENCIÓN,...,0,0,0,0,NaN,NaN,1,ND,8.170379,-76.059697
68599,9909,NaN,1996,1,13,27006,ACANDI,CHOCO,URABA,FALSO POSITIVO,...,0,1,0,0,NaN,NaN,1,ARTEFACTO EXPLOSIVO,8.511406,-77.280098
68600,99394,NaN,2000,7,17,23855,VALENCIA,CORDOBA,URABA,ASALTO,...,0,0,0,1,NaN,NaN,1,ARMA CORTO PUNZANTE,8.108959,-76.345340


In [15]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
DF_filtered = DF[relevant_columns]

# Group by the required keys
CasosDF = DF_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosDF[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosDF.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,1850,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1961,3,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1963,1,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1964,1,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1965,3,0,0,0,0,0


In [16]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
DF_filtered = DF[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    DF_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    DF_filtered.loc[DF_filtered['Presunto Responsable'] == original, new] = DF_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelDF = DF_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2081874655.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2081874655.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2081874655.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [17]:
MA = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosMA_202309.xlsx', dtype={'Código DANE de Municipio': str})
MA

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Extorsión,Lesionados Civiles,Pillaje,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total de Víctimas del Caso,Tipo de Armas,Latitud,Longitud
0,100649,CR030032,1998,1,25,25530,PARATEBUENO,CUNDINAMARCA,PIEDEMONTE LLANERO,RUTA,...,1,0,0,2,NaN,VENTA A BAJO PRECIO.,5,ARMA DE FUEGO,4.374833,-73.212827
1,102446,NaN,2007,10,17,27361,ISTMINA,CHOCO,VALLE DE SAN JUAN,ENGAÑO,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,5.153946,-76.685180
2,105517,NaN,2005,12,7,44430,MAICAO,LA GUAJIRA,ALTA GUAJIRA,ASALTO,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,11.378327,-72.241905
3,105564,NaN,2000,6,18,44430,MAICAO,LA GUAJIRA,ALTA GUAJIRA,SICARIATO,...,0,3,0,0,NaN,NaN,4,ARMA DE FUEGO,11.378327,-72.241905
4,105630,NaN,2002,6,2,44430,MAICAO,LA GUAJIRA,ALTA GUAJIRA,SICARIATO,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,11.378327,-72.241905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319,95261,NaN,1996,12,26,20060,BOSCONIA,CESAR,SIERRA NEVADA DE SANTA MARTA,RETENCIÓN/EJECUCIÓN,...,0,0,0,0,NaN,NaN,5,ND,9.975095,-73.888768
4320,95566,NaN,1998,9,9,20400,LA JAGUA DE IBIRICO,CESAR,SERRANIA DEL PERIJA,REUNIÓN PÚBLICA,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,9.563887,-73.334228
4321,96825,NaN,2002,9,10,20750,SAN DIEGO,CESAR,SERRANIA DEL PERIJA,RETENCIÓN/EJECUCIÓN,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,10.244292,-73.144730
4322,98130,NaN,2002,10,31,23466,MONTELIBANO,CORDOBA,ALTO SINU Y SAN JORGE,RETENCIÓN/EJECUCIÓN,...,0,0,0,0,NaN,NaN,4,ARMA DE FUEGO,7.973777,-75.416818


In [18]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
MA_filtered = MA[relevant_columns]

# Group by the required keys
CasosMA = MA_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosMA[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosMA.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,18,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1959,11,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1961,6,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1962,11,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1963,23,0,0,0,0,0


In [19]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
MA_filtered = MA[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    MA_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    MA_filtered.loc[MA_filtered['Presunto Responsable'] == original, new] = MA_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelMA = MA_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()



/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3055315682.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MA_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3055315682.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MA_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/3055315682.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [20]:
MI = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosMI_202309.xlsx', dtype={'Código DANE de Municipio': str})
MI

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Otro Hecho Simultáneo,Total Combatientes,Total Civiles,Total Civiles y Combatientes,Total de Víctimas del Caso,Tipo de Armas,Tipo de Evento,Total Civiles y Combatientes Heridos,Latitud,Longitud
0,113698,NaN,2012,1,31,50325,MAPIRIPAN,META,ALTILLANURA,ATENTADO,...,NaN,1,1,1,1,ARTEFACTO EXPLOSIVO,ATENTADO POR MAP,0,2.896439,-72.135645
1,116764,NaN,2012,8,10,52835,SAN ANDRES DE TUMACO,NARIÑO,ANDEN PACIFICO SUR,ATENTADO,...,NaN,0,1,1,1,MINAS ANTIPERSONAL MAP,ATENTADO POR MAP,0,1.807117,-78.763854
2,12200,CR201712202,1998,3,18,05001,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,ATENTADO,...,NaN,1,0,1,1,MINAS ANTIPERSONAL MAP,ATENTADO POR MAP,0,6.246716,-75.581713
3,139902,NaN,2005,1,24,81794,TAME,ARAUCA,SARARE,ATENTADO,...,NaN,0,1,1,1,ARTEFACTOS EXPLOSIVOS IMPROVISADOS AEI,ATENTADO POR AIE,0,6.449379,-71.758611
4,175352,NaN,1998,11,13,86568,PUERTO ASIS,PUTUMAYO,BAJO PUTUMAYO,ATENTADO,...,NaN,0,2,2,2,ARTEFACTO EXPLOSIVO,ATENTADO POR AIE,0,0.594663,-76.566938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,744559,NaN,2023,8,17,27150,CARMEN DEL DARIEN,CHOCO,URABA,INCIDENTE,...,NaN,0,0,0,0,ARTEFACTOS EXPLOSIVOS IMPROVISADOS AEI,DESMINADO MILITAR EN OPERACIONES,0,7.178538,-77.033191
9924,744562,NaN,2023,8,17,05887,YARUMAL,ANTIOQUIA,NORTE DE ANTIOQUIA,INCIDENTE,...,NaN,0,0,0,0,ARTEFACTOS EXPLOSIVOS IMPROVISADOS AEI,DESMINADO MILITAR EN OPERACIONES,0,6.963872,-75.418833
9925,754573,NaN,2023,8,24,54810,TIBU,NORTE DE SANTANDER,CATATUMBO,INCIDENTE,...,NaN,0,0,0,0,MINAS ANTIPERSONAL MAP,DESMINADO MILITAR EN OPERACIONES,0,8.639549,-72.735919
9926,754592,NaN,2023,1,26,50325,MAPIRIPAN,META,ALTILLANURA,INCIDENTE,...,NaN,0,0,0,0,MINAS ANTIPERSONAL MAP,DESMINADO MILITAR EN OPERACIONES,0,2.896439,-72.135645


In [21]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
MI_filtered = MI[relevant_columns]

# Group by the required keys
CasosMI = MI_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosMI[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosMI.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,13,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1993,1,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1997,1,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2000,1,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,2001,2,0,0,0,0,0


In [22]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
MI_filtered = MI[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    MI_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    MI_filtered.loc[MI_filtered['Presunto Responsable'] == original, new] = MI_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelMI = MI_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()


/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/4136976572.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MI_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/4136976572.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MI_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/4136976572.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [23]:
RU = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosRU_202309.xlsx', dtype={'Código DANE de Municipio': str})
RU

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Lesionados Civiles,Pillaje,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total de Víctimas del Caso,Forma de Vinculación,Tipo de Vinculación,Latitud,Longitud
0,207556,NaN,2001,4,25,05756,SONSON,ANTIOQUIA,ORIENTE ANTIOQUEÑO,PERSUASIÓN,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,5.837681,-74.828073
1,209268,NaN,0,0,0,05234,DABEIBA,ANTIOQUIA,OCCIDENTE ANTIOQUEÑO,DESCONOCIDA,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,7.000322,-76.262977
2,225057,NaN,1988,0,0,68689,SAN VICENTE DE CHUCURI,SANTANDER,MAGDALENA MEDIO,COACCIÓN,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,6.880383,-73.411024
3,228661,NaN,1998,2,15,66456,MISTRATO,RISARALDA,EJE CAFETERO,PERSUASIÓN,...,0,0,0,NaN,OCULTAMIENTO DE CADÁVER,1,INDIVIDUAL,RECLUTAMIENTO,5.416268,-75.904326
4,229622,NaN,1993,1,11,50001,VILLAVICENCIO,META,PIEDEMONTE LLANERO,ENGAÑO,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,4.123861,-73.627092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16895,744280,NaN,2000,8,0,66687,SANTUARIO,RISARALDA,EJE CAFETERO,PERSUASIÓN,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,5.072781,-75.962116
16896,744281,NaN,2002,0,0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,DESCONOCIDA,...,0,0,0,NaN,NaN,1,SIN INFORMACIÓN,RECLUTAMIENTO,4.000000,-72.000000
16897,754591,NaN,2023,1,31,18756,SOLANO,CAQUETA,AMAZONIA SUR-ORIENTAL,COACCIÓN,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,0.698978,-75.253704
16898,754699,NaN,2023,9,1,05670,SAN ROQUE,ANTIOQUIA,NORDESTE ANTIOQUEÑO,DESCONOCIDA,...,0,0,0,NaN,NaN,1,INDIVIDUAL,RECLUTAMIENTO,6.485949,-75.019158


In [24]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
RU_filtered = RU[relevant_columns]

# Group by the required keys
CasosRU = RU_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosRU[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosRU.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,1917,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1962,1,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1978,1,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1979,1,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1983,2,0,0,0,0,0


In [25]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
RU_filtered = RU[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    RU_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    RU_filtered.loc[RU_filtered['Presunto Responsable'] == original, new] = RU_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelRU = RU_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1903967586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RU_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1903967586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RU_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1903967586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [26]:
SE = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosSE_202309.xlsx', dtype={'Código DANE de Municipio': str})
SE

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total de Víctimas del Caso,Modalidad de Secuestro,Tipo de Secuestro,Finalidad del Secuestro,Exigencia para la Liberación,Latitud,Longitud
0,102517,CR2017292221,2005,8,2,27001,QUIBDO,CHOCO,ATRATO,RETÉN,...,1,NaN,SEÑALAMIENTO,1,INDIVIDUAL,ND,ND,ND,5.875260,-76.583609
1,103666,NaN,2014,7,13,41359,ISNOS,HUILA,SUR DEL HUILA,ASALTO,...,0,NaN,NaN,1,INDIVIDUAL,EXTORSIVO,ECONÓMICA,CUANTÍA,1.929467,-76.217637
2,104360,NaN,2006,12,21,41668,SAN AGUSTIN,HUILA,MACIZO COLOMBIANO,DESCONOCIDA,...,0,NaN,NaN,1,INDIVIDUAL,EXTORSIVO,ECONÓMICA,CUANTÍA,1.881081,-76.270360
3,104504,NaN,1975,6,19,44001,RIOHACHA,LA GUAJIRA,SIERRA NEVADA DE SANTA MARTA,DESCONOCIDA,...,0,NaN,NaN,1,INDIVIDUAL,SIMPLE,POLÍTICA,ND,11.528751,-72.911489
4,110951,NaN,2005,9,26,47745,SITIONUEVO,MAGDALENA,CIENAGA GRANDE DE SANTA MARTA,RETENCIÓN/EJECUCIÓN,...,1,NaN,NaN,1,INDIVIDUAL,ND,ND,ND,10.775410,-74.720169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31384,88696,NaN,2003,4,16,18860,VALPARAISO,CAQUETA,FLORENCIA Y AREA DE INFLUENCIA,ASALTO,...,0,NaN,NaN,1,INDIVIDUAL,SIMPLE,ND,ND,1.194583,-75.706584
31385,92152,CR201792152,2004,1,28,20001,VALLEDUPAR,CESAR,SIERRA NEVADA DE SANTA MARTA,DESCONOCIDA,...,0,NaN,NaN,3,COLECTIVO,ND,ND,ND,10.460481,-73.259389
31386,93243,NaN,2003,3,27,20011,AGUACHICA,CESAR,MAGDALENA MEDIO,INTERCEPTACIÓN,...,0,NaN,EXILIO - INTENTO DE RECLUTAMIENTO,2,MÚLTIPLE,SIMPLE,ND,ND,8.310337,-73.599029
31387,95471,NaN,1982,8,9,20295,GAMARRA,CESAR,MAGDALENA MEDIO,DESCONOCIDA,...,0,NaN,NaN,2,MÚLTIPLE,EXTORSIVO,ECONÓMICA,CUANTÍA,8.320449,-73.743760


In [27]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
SE_filtered = SE[relevant_columns]

# Group by the required keys
CasosSE = SE_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosSE[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosSE.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,1220,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1958,2,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1962,3,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1964,2,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1967,1,0,0,0,0,0


In [28]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
SE_filtered = SE[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    SE_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    SE_filtered.loc[SE_filtered['Presunto Responsable'] == original, new] = SE_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelSE = SE_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/234051309.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SE_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/234051309.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SE_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/234051309.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [29]:
VS = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/1:_RawData/Violence (Cases)/CasosVS_202309.xlsx', dtype={'Código DANE de Municipio': str})
VS

/Users/edmundoarias/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ID Caso,ID Caso Relacionado,Año,Mes,Día,Código DANE de Municipio,Municipio,Departamento,Región,Modalidad,...,Extorsión,Lesionados Civiles,Pillaje,Tortura,Violencia Basada en Género,Otro Hecho Simultáneo,Total de Víctimas del Caso,Tipo de Armas,Latitud,Longitud
0,235240,CR040054,1997,12,3,17614,RIOSUCIO,CALDAS,EJE CAFETERO,RETENCIÓN/EJECUCIÓN,...,0,0,0,0,NaN,SEVICIA,1,ARMA CORTO PUNZANTE,5.423681,-75.702070
1,251906,CR021706,2001,6,10,68615,RIONEGRO,SANTANDER,MAGDALENA MEDIO,ASALTO,...,0,0,0,2,NaN,NaN,2,ARMA DE FUEGO,7.265010,-73.150147
2,359623,NaN,0,0,0,20570,PUEBLO BELLO,CESAR,SIERRA NEVADA DE SANTA MARTA,DESCONOCIDA,...,0,0,0,0,NaN,NaN,1,ND,10.415062,-73.589011
3,359624,NaN,0,0,0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,COERCIÓN,...,0,0,0,0,NaN,NaN,1,ND,4.000000,-72.000000
4,359625,CR004865,0,0,0,54000,SIN INFORMACION,NORTE DE SANTANDER,NaN,COERCIÓN,...,0,0,0,0,NaN,NaN,1,ND,8.095138,-72.881883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16473,743829,NaN,1963,12,13,20013,AGUSTIN CODAZZI,CESAR,SERRANIA DEL PERIJA,ATAQUE INDISCRIMINADO,...,0,0,0,0,NaN,NaN,1,ARMA DE FUEGO,10.040458,-73.238396
16474,743898,NaN,1961,3,0,41020,ALGECIRAS,HUILA,CENTRO DEL HUILA,ASALTO,...,0,0,0,0,NaN,NaN,1,ND,2.522448,-75.315373
16475,743900,NaN,1961,3,0,73352,ICONONZO,TOLIMA,SUMAPAZ,RETENCIÓN,...,0,0,0,0,NaN,NaN,2,ND,4.176436,-74.531976
16476,743996,NaN,2002,11,26,05088,BELLO,ANTIOQUIA,VALLE DE ABURRA,RETENCIÓN,...,0,0,0,1,VIOLENCIA FÍSICA,ND,1,ND,6.334069,-75.554665


In [30]:
relevant_columns = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año', 'Total de Víctimas del Caso']
VS_filtered = VS[relevant_columns]

# Group by the required keys
CasosVS = VS_filtered.groupby(['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']).sum().reset_index()

# Placeholder for perpetrator columns, assuming a direct method to categorize them will be applied later
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    CasosVS[perpetrator] = 0  # Placeholder values

# Display the transformed but incomplete dataset structure for review
CasosVS.head()

,Código DANE de Municipio,Municipio,Departamento,Región,Año,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,VictimasEstado,VictimasArmado,VictimasDesmov
0,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,0,233,0,0,0,0,0
1,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1961,1,0,0,0,0,0
2,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1978,1,0,0,0,0,0
3,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1989,3,0,0,0,0,0
4,00000,SIN INFORMACION,SIN INFORMACION,SIN INFORMACION,1991,3,0,0,0,0,0


In [31]:
# Include "Descripción Grupo Armado 2" in the filtered data
relevant_columns.append('Presunto Responsable')
VS_filtered = VS[relevant_columns]

# Map "Descripción Grupo Armado 2" to the specified perpetrator categories
# First, initialize the perpetrator columns with 0
for perpetrator in ['VictimasGuerrilla', 'VictimasParas', 'VictimasEstado', 'VictimasArmado', 'VictimasDesmov']:
    VS_filtered[perpetrator] = 0

# Define the mapping from "Descripción Grupo Armado 2" to the new columns
perpetrator_mapping = {
    'GUERRILLA': 'VictimasGuerrilla',
    'GRUPO PARAMILITAR': 'VictimasParas',
    'AGENTE DEL ESTADO': 'VictimasEstado',
    'GRUPO ARMADO NO IDENTIFICADO': 'VictimasArmado',
    'GRUPO POSDESMOVILIZACIÓN': 'VictimasDesmov'
}

# Apply the mapping for each row
for original, new in perpetrator_mapping.items():
    # Allocate the "Total de Víctimas del Caso" based on the perpetrator
    VS_filtered.loc[VS_filtered['Presunto Responsable'] == original, new] = VS_filtered['Total de Víctimas del Caso']

# Now group by the specified keys and sum up the victims for each perpetrator category
group_keys = ['Código DANE de Municipio', 'Municipio', 'Departamento', 'Región', 'Año']
PanelVS = VS_filtered.groupby(group_keys).agg({
    'Total de Víctimas del Caso': 'sum',
    'VictimasGuerrilla': 'sum',
    'VictimasParas': 'sum',
    'VictimasEstado': 'sum',
    'VictimasArmado': 'sum',
    'VictimasDesmov': 'sum'
}).reset_index()

/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2761859286.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VS_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2761859286.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VS_filtered[perpetrator] = 0
/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/2761859286.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

# Panel

In [32]:
# List all municipios – DANE Codes
codes = pd.read_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Project/1:_RawData/CodMunicipiosDANE.xlsx', dtype={'Codmun': str})

In [33]:
# Set relevant years to study
years = pd.DataFrame({'Año': np.arange(1955, 2023)})

# Crear el conjunto de datos de panel cruzando los municipios con los años
# Usamos una técnica de 'merge' para crear el producto cartesiano
panel_data = codes.assign(key=1).merge(years.assign(key=1), on='key').drop('key', axis=1)
panel_data['Codmun'] = panel_data['Codmun'].astype(str)

In [34]:
panel_data.rename(columns={'Codmun': 'Código DANE de Municipio'}, inplace=True)

### Merge all

In [35]:
def ensure_string_column(dataframes, column_name):
    for df in dataframes:
        df[column_name] = df[column_name].astype(str)

dataframes = [PanelAB, PanelAP, PanelAS, PanelAT, PanelDF, PanelMA, PanelMI, PanelRU, PanelSE, PanelVS ]
ensure_string_column(dataframes, 'Código DANE de Municipio')

In [36]:
m1 = pd.merge(panel_data, PanelAB, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_AB'))
m1.fillna(0, inplace=True)

m2 = pd.merge(m1, PanelAP, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_AP'))
m2.fillna(0, inplace=True)

m3 = pd.merge(m2, PanelAS, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_AS'))
m3.fillna(0, inplace=True)

m4 = pd.merge(m3, PanelAT, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_AT'))
m4.fillna(0, inplace=True)

m5 = pd.merge(m4, PanelDF, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_DF'))
m5.fillna(0, inplace=True)

m6 = pd.merge(m5, PanelMA, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_MA'))
m6.fillna(0, inplace=True)

m7 = pd.merge(m6, PanelMI, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_MI'))
m7.fillna(0, inplace=True)

m8 = pd.merge(m7, PanelRU, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_RU'))
m8.fillna(0, inplace=True)

m9 = pd.merge(m8, PanelSE, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_SE'))
m9.fillna(0, inplace=True)

m10 = pd.merge(m9, PanelVS, on=["Código DANE de Municipio", "Año"], how='left', suffixes=('', '_VS'))
m10.fillna(0, inplace=True)

panel_final = m10
panel_final



,Departamento,Municipio,Código DANE de Municipio,Año,Municipio_AB,Departamento_AB,Región,Total de Víctimas del Caso,VictimasGuerrilla,VictimasParas,...,VictimasDesmov_SE,Municipio_VS,Departamento_VS,Región_VS,Total de Víctimas del Caso_VS,VictimasGuerrilla_VS,VictimasParas_VS,VictimasEstado_VS,VictimasArmado_VS,VictimasDesmov_VS
0,Antioquia,Medellín,05001,1955,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,Antioquia,Medellín,05001,1956,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Antioquia,Medellín,05001,1957,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Antioquia,Medellín,05001,1958,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,Antioquia,Medellín,05001,1959,MEDELLIN,ANTIOQUIA,VALLE DE ABURRA,1.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74999,Guainía,Barrancominas,94343,2018,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
75000,Guainía,Barrancominas,94343,2019,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
75001,Guainía,Barrancominas,94343,2020,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
75002,Guainía,Barrancominas,94343,2021,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### Sum & Clean

In [37]:
panel_final.columns

Index(['Departamento', 'Municipio', 'Código DANE de Municipio', 'Año',
       'Municipio_AB', 'Departamento_AB', 'Región',
       'Total de Víctimas del Caso', 'VictimasGuerrilla', 'VictimasParas',
       'VictimasEstado', 'VictimasArmado', 'VictimasDesmov', 'Municipio_AP',
       'Departamento_AP', 'Región_AP', 'Total de Víctimas del Caso_AP',
       'VictimasGuerrilla_AP', 'VictimasParas_AP', 'VictimasEstado_AP',
       'VictimasArmado_AP', 'VictimasDesmov_AP', 'Municipio_AS',
       'Departamento_AS', 'Región_AS', 'Total de Víctimas del Caso_AS',
       'VictimasGuerrilla_AS', 'VictimasParas_AS', 'VictimasEstado_AS',
       'VictimasArmado_AS', 'VictimasDesmov_AS', 'Municipio_AT',
       'Departamento_AT', 'Región_AT', 'Total de Víctimas del Caso_AT',
       'VictimasGuerrilla_AT', 'VictimasParas_AT', 'VictimasEstado_AT',
       'VictimasArmado_AT', 'VictimasDesmov_AT', 'Municipio_DF',
       'Departamento_DF', 'Región_DF', 'Total de Víctimas del Caso_DF',
       'VictimasGuerrilla_

In [41]:
# Violencia (Agg)
panel_final['Violencia'] = panel_final[['Total de Víctimas del Caso', 'Total de Víctimas del Caso_AP', 'Total de Víctimas del Caso_AS', 'Total de Víctimas del Caso_AT', 'Total de Víctimas del Caso_DF', 'Total de Víctimas del Caso_MA', 'Total de Víctimas del Caso_MI', 'Total de Víctimas del Caso_RU', 'Total de Víctimas del Caso_SE', 'Total de Víctimas del Caso_VS']].sum(axis=1)

# Violencia Guerrilla
panel_final['V_Guerrilla'] = panel_final[['VictimasGuerrilla', 'VictimasGuerrilla_AP', 'VictimasGuerrilla_AS', 'VictimasGuerrilla_AT', 'VictimasGuerrilla_DF', 'VictimasGuerrilla_MA', 'VictimasGuerrilla_MI', 'VictimasGuerrilla_RU', 'VictimasGuerrilla_SE', 'VictimasGuerrilla_VS']].sum(axis=1)

# Violencia Estado
panel_final['V_Estado'] = panel_final[['VictimasEstado', 'VictimasEstado_AP', 'VictimasEstado_AS', 'VictimasEstado_AT', 'VictimasEstado_DF', 'VictimasEstado_MA', 'VictimasEstado_MI', 'VictimasEstado_RU', 'VictimasEstado_SE', 'VictimasEstado_VS']].sum(axis=1)

# Violencia Paramilitar
panel_final['V_Paramilitar'] = panel_final[['VictimasParas', 'VictimasParas_AP', 'VictimasParas_AS', 'VictimasParas_AT', 'VictimasParas_DF', 'VictimasParas_MA', 'VictimasParas_MI', 'VictimasParas_RU', 'VictimasParas_SE', 'VictimasParas_VS']].sum(axis=1)

# Violencia Desmovilizados
panel_final['V_Desmovilizados'] = panel_final[['VictimasDesmov', 'VictimasDesmov_AP', 'VictimasDesmov_AS', 'VictimasDesmov_AT', 'VictimasDesmov_DF', 'VictimasDesmov_MA', 'VictimasDesmov_MI', 'VictimasDesmov_RU', 'VictimasDesmov_SE', 'VictimasDesmov_VS']].sum(axis=1)

# Violencia Grupos Armados
panel_final['V_GruposArma'] = panel_final[['VictimasArmado', 'VictimasArmado_AP', 'VictimasArmado_AS', 'VictimasArmado_AT', 'VictimasArmado_DF', 'VictimasArmado_MA', 'VictimasArmado_MI', 'VictimasArmado_RU', 'VictimasArmado_SE', 'VictimasArmado_VS']].sum(axis=1)

In [42]:
# Columns to keep
keep =['Departamento', 'Municipio', 'Código DANE de Municipio', 'Año', 'Violencia', 'V_Guerrilla', 'V_Estado', 'V_Paramilitar', 'V_Desmovilizados', 'V_GruposArma']
Panel_v3 = panel_final[keep]

# Rename
Panel_v3.rename(columns={'Código DANE de Municipio': 'id', 'Año': 'Periodo'}, inplace=True)


/var/folders/4q/634d8_qs2qdd58rzydjvszzm0000gn/T/ipykernel_97086/1591269542.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Panel_v3.rename(columns={'Código DANE de Municipio': 'id', 'Año': 'Periodo'}, inplace=True)


In [49]:
# Check to see what municipios never suffered any type of violence
violence_sum_by_municipality = Panel_v3.groupby('id')['V_GruposArma'].sum()

# Identify municipalities with a sum of 0 (never suffered violence)
municipalities_never_suffered_violence = violence_sum_by_municipality[violence_sum_by_municipality == 0]

# Calculate the percentage
percentage_never_suffered = (len(municipalities_never_suffered_violence) / Panel_v3['id'].nunique()) * 100

percentage_never_suffered

13.417951042611062

## Export data

In [52]:
Panel_v3.to_excel('/Users/edmundoarias/Documents/Uniandes/2024-10/HEC/Political-Attitudes/2:_ProcessedData/MergedData/Violence/PanelMunicipios_Violencia Extended.xlsx')